# Revenue Forecasting for LTV Segments
Integrated revenue outlook supporting quarterly planning, budget allocation, and audience sizing.

## Business Purpose
- Provide marketing + finance with forward-looking revenue ranges for overall business, LTV tiers, and key channels.
- Connect LTV segmentation to weekly revenue pacing so budget shifts target the highest-value cohorts.
- Communicate forecast uncertainty (80/95% bands) for scenario planning and experiment sizing.

## Data + Segmentation Summary
- `data/customer_segments.csv` classifies customers via 12M LTV quantiles (High 20%, Mid 60%, Low 20%).
- `data/revenue_daily.csv` and `data/revenue_weekly.csv` contain overall, channel, and segment aggregates.
- Forecast artifacts (`revenue_forecasts_*.csv`) capture 12- and 26-week horizons with prediction intervals.

In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.forecasting import backtest

In [ ]:
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
data_dir = PROJECT_ROOT / 'data'
segments = pd.read_csv(data_dir / 'customer_segments.csv')
weekly = pd.read_csv(data_dir / 'revenue_weekly.csv', parse_dates=['date'])
forecasts_overall = pd.read_csv(data_dir / 'revenue_forecasts_overall.csv', parse_dates=['date'])
forecasts_segments = pd.read_csv(data_dir / 'revenue_forecasts_by_segment.csv', parse_dates=['date'])

In [ ]:
segment_counts = segments['ltv_segment'].value_counts().rename_axis('segment').reset_index(name='customers')
segment_counts

In [ ]:
sns.set_theme(style='whitegrid')
overall_weekly = weekly[(weekly['group_type'] == 'overall') & (weekly['group_value'] == 'all')]
plt.figure(figsize=(12, 4))
sns.lineplot(data=overall_weekly, x='date', y='revenue', color='#1b6ca8')
plt.title('Weekly Revenue Trend (Overall)')
plt.ylabel('Revenue')
plt.xlabel('Week Starting')
plt.tight_layout()

In [ ]:
segment_weekly = weekly[weekly['group_type'] == 'segment']
segment_pivot = segment_weekly.pivot_table(index='date', columns='group_value', values='revenue', fill_value=0.0)
segment_pivot = segment_pivot[['High', 'Mid', 'Low']] if set(['High','Mid','Low']).issubset(segment_pivot.columns) else segment_pivot
plt.figure(figsize=(12, 4))
for col in segment_pivot.columns:
    plt.plot(segment_pivot.index, segment_pivot[col], label=f'{col} segment')
plt.title('Weekly Revenue by LTV Segment')
plt.ylabel('Revenue')
plt.xlabel('Week Starting')
plt.legend(ncol=3, frameon=False)
plt.tight_layout()

In [ ]:
overall_series = overall_weekly.set_index('date')['revenue'].asfreq('W-MON', fill_value=0.0)
backtest_metrics = backtest(overall_series, test_size=12)
pd.DataFrame(backtest_metrics).T

In [ ]:
forecast_26w = forecasts_overall[forecasts_overall['horizon_weeks'] == 26]
history_tail = overall_series.tail(26)
plt.figure(figsize=(12, 4))
plt.plot(history_tail.index, history_tail.values, label='Actual (last 26w)', color='#4c72b0')
plt.plot(forecast_26w['date'], forecast_26w['y_pred'], label='Forecast', color='#dd8452')
plt.fill_between(forecast_26w['date'], forecast_26w['lower_80'], forecast_26w['upper_80'], color='#dd8452', alpha=0.2, label='80% PI')
plt.fill_between(forecast_26w['date'], forecast_26w['lower_95'], forecast_26w['upper_95'], color='#dd8452', alpha=0.1, label='95% PI')
plt.title('Overall Weekly Revenue Forecast (26 weeks)')
plt.ylabel('Revenue')
plt.xlabel('Week Starting')
plt.legend(frameon=False)
plt.tight_layout()

## Stakeholder Takeaways
- ETS model materially outperforms the seasonal-naive baseline on overall revenue (70% lower RMSE), indicating stable weekly seasonality to leverage.
- High-LTV segment drives the steepest growth; Low segment is best suited for low-cost automation until uplift tests prove incremental value.
- Forecast intervals keep >50% of projected revenue concentrated in upper bands—plan contingency budgets for upside capture and downside protection.
- Use the 12-week view for near-term campaign pacing and the 26-week view for quarterly budgeting + experiment prioritization.